In [1]:
import pandas as pd


In [ ]:
data_mentor = pd.read_csv('data-mentors.csv', delimiter='\s+', header=None)

In [8]:
data_pairing = pd.read_csv('data-pairings.csv', delimiter='\s+', header=None)
data_pairing


,0,1
0,"Mentors,Students,Second",Students
1,"Mentor1,Student18,None",NaN
2,"Mentor2,Student86,None",NaN
3,"Mentor3,Student45,None",NaN
4,"Mentor4,Student10,None",NaN
5,"Mentor5,Student19,Student80",NaN
6,"Mentor6,Student49,None",NaN
7,"Mentor7,Student14,None",NaN
8,"Mentor8,Student92,None",NaN
9,"Mentor9,Student84,None",NaN
